# POPULARITY BASED RECOMMENDOR SYSTEM

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np

song_df_1 = pd.read_table(r'C:\Users\sv784\Desktop\SongRecommendor\triplet_file.txt', header = None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']
song_df_1.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFGUAY12AB017B0A8,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOFRQTD12A81C233C0,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOHQWYZ12A6D4FA701,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIYTOA12A6D4F9A23,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5


In [3]:
len(song_df_1)

20024

In [4]:
song_df_2 = pd.read_excel(r'C:\Users\sv784\Desktop\SongRecommendor\metadata_file.xlsx')
song_df_2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,KarkuteillÃ¤,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos QuerÃ©s,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [5]:
#merging both the files
song_df = song_df_2.merge(song_df_1, on = 'song_id')
song_df.head()

,song_id,title,release,artist_name,year,user_id,listen_count
0,SOSZNRJ12A8AE46E38,Anyone Else But You,Juno - Music From The Motion Picture,Michael Cera & Ellen Page,2007,d6589314c0a9bcbca4fee0c93b14bc402363afea,6
1,SOSZNRJ12A8AE46E38,Anyone Else But You,Juno - Music From The Motion Picture,Michael Cera & Ellen Page,2007,484b69dd013df1ec0cfd504886d4f647cb32b08f,1
2,SOSZNRJ12A8AE46E38,Anyone Else But You,Juno - Music From The Motion Picture,Michael Cera & Ellen Page,2007,3f9ed694a79835c921ef6d94acd28f876c1d901e,4
3,SOGKGLB12A81C22AFA,Drunk and Hot Girls,Graduation,Kanye West / Mos Def,2007,529b42cdbc379ad2e765aec6d3bad8a192038741,2
4,SOGKGLB12A81C22AFA,Drunk and Hot Girls,Graduation,Kanye West / Mos Def,2007,73e9e981d5fc2a6453cdfb5025e2fa0a144a4142,1


In [6]:
print('TextFile:', song_df_1.shape)
print('ExcelFile:', song_df_2.shape)
print('MergedFile:', song_df.shape)

TextFile: (20024, 3)
ExcelFile: (10000, 5)
MergedFile: (179, 7)


In [7]:
len(song_df)

179

Now we will count the total listen count of each song and arrange data in decreasing order.

In [8]:
#Arranging data according to number of times they are played.
song_group = song_df.groupby(['song_id']).agg({'listen_count':'count'}).reset_index()
group_sum = song_group['listen_count'].sum()
song_group['percentage'] = song_group['listen_count'].div(group_sum)*100

#grouping them in ascending order
song_group.sort_values(['listen_count', 'song_id'], ascending = [0,1]).head()

,song_id,listen_count,percentage
6,SOBOUPA12A6D4F81F1,21,11.731844
32,SOMMONH12A6D4F41CD,9,5.027933
13,SOESZWS12A6D4F910C,6,3.351955
14,SOETSTN12A81C219A9,6,3.351955
21,SOGLBQA12A6701E7AF,6,3.351955


In [9]:
#Number of unique users
users = song_df['user_id'].unique()
len(users)

140

In [10]:
#Number of unique song
song = song_df['song_id'].unique()
len(song)

77

In [11]:
#Splitting our data in two parts for testing and training purpose.
data_training, data_testing = train_test_split(song_df, test_size = 0.40, random_state = 0)
print(data_training.head())

               song_id                                              title  \
97  SOPKTEV12A8C141E33                                         Burnin' Up   
43  SOBOUPA12A6D4F81F1                             SinceritÃ© Et Jalousie   
10  SOGELNH12AB017F92C                             Love vs. Money: Part 2   
93  SOOWOSS12A67ADE2D3  Rebirth of Slick (Cool Like Dat) (2005 Digital...   
73  SOLPVAQ12AB017EB35                                  All Men Are Liars   

                                            release      artist_name  year  \
97                             2009 GRAMMY Nominees   Jonas Brothers  2008   
43                                  Simple Et Funky  Alliance Ethnik     0   
10                                    Love Vs Money        The-Dream     0   
93  Beyond The Spectrum - The Creamy Spy Chronicles  Digable Planets     0   
73        Quiet Please... The New Best Of Nick Lowe        Nick Lowe     0   

                                     user_id  listen_count  
97  469

# Our popularity based recommendor Class


Will randomly take 40% of our data as training data and train our model with it.

In [12]:
class popularity_based_recommendor():
    def _init_(self):
        self.data_training = None
        self.user_id = None
        self.song_id = None
        self.popular_recommendations = None
    
    #Popularity based recommedation model 
    def create(self, data_training, user_id, song_id):
        self.data_training = data_training
        self.user_id = user_id
        self.song_id = song_id
        
        #counting user_id for each unique song_id.
        data_training_group = data_training.groupby([self.song_id]).agg({self.user_id:'count'}).reset_index()
        #Setting count as recommendation score.
        data_training_group.rename(columns = {'user_id': 'score'}, inplace = True)
        
        #Sorting song based upon recommendation score.
        training_data_sort = data_training_group.sort_values(['score', self.song_id], ascending = [0, 1])
        
        #Generating recommendation rank based on recommendation score.
        training_data_sort['Rank'] = training_data_sort['score'].rank(ascending = 0, method = 'first')
        
        #Will get the top 5 recommendation
        self.popular_recommendations = training_data_sort.head(5)
        
    def popularity_based_recommendation(self, user_id):
        recommendation_forUser = self.popular_recommendations
        recommendation_forUser['user_id'] = user_id
        
        return recommendation_forUser

Creating an instance of recommendor class.

In [13]:
class_instance = popularity_based_recommendor()
class_instance.create(data_training, 'user_id', 'title')

# Final results

Results will not differ for different users. 

In [14]:
#Predictions for first user
user_id = users[10]
class_instance.popularity_based_recommendation(user_id).head(5)

,title,score,Rank,user_id
41,SinceritÃ© Et Jalousie,14,1.0,9b887e10a4711486085c4fae2d2599fc0d2c484d
47,The Maestro,6,2.0,9b887e10a4711486085c4fae2d2599fc0d2c484d
0,All Men Are Liars,4,3.0,9b887e10a4711486085c4fae2d2599fc0d2c484d
40,Silent Shout,4,4.0,9b887e10a4711486085c4fae2d2599fc0d2c484d
50,Too Much Love,4,5.0,9b887e10a4711486085c4fae2d2599fc0d2c484d


Making prediction for another user and result should remain same. 

In [15]:
#Predictions for second user
user_id = users[5]
class_instance.popularity_based_recommendation(user_id).head(10)

,title,score,Rank,user_id
41,SinceritÃ© Et Jalousie,14,1.0,1b704d4cddabea8258bd93497fcb73eab32fa592
47,The Maestro,6,2.0,1b704d4cddabea8258bd93497fcb73eab32fa592
0,All Men Are Liars,4,3.0,1b704d4cddabea8258bd93497fcb73eab32fa592
40,Silent Shout,4,4.0,1b704d4cddabea8258bd93497fcb73eab32fa592
50,Too Much Love,4,5.0,1b704d4cddabea8258bd93497fcb73eab32fa592
